In [1]:
import gymnasium as gym
import gymnasium_robotics

# PyTorch
import torch

# from collections import deque
import numpy as np
from stable_baselines3 import DDPG
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv

In [2]:
env_id = 'FrankaKitchen-v1'
task = 'kettle'
gym.register_envs(gymnasium_robotics)

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
n_actions = 9

In [5]:
flat_dim = 73
obs_low = np.full((flat_dim,), -1e10, dtype=np.float32)
obs_high = np.full((flat_dim,), 1e10, dtype=np.float32)

class FlattenDictWrapper(gym.ObservationWrapper):    
    def __init__(self, env):
        super().__init__(env)
        self.keys = env.observation_space.spaces.keys()
        self.observation_space = gym.spaces.Box(low=obs_low, high=obs_high, shape=(flat_dim,), dtype=np.float32)

    def observation(self, observation):
        achieved = observation['achieved_goal'][task].astype(np.float32)
        desired = observation['desired_goal'][task].astype(np.float32)
        obs = observation['observation'].astype(np.float32)

        flat_obs = np.concatenate([achieved, desired, obs], dtype=np.float32)
        return flat_obs

In [6]:
# import os
# os.environ["MUJOCO_MODEL_PATH"] = "c:\\Users\\pc\\Documents\\repos\\Gymnasium-Robotics\\gymnasium_robotics\\envs\\assets"


In [7]:
def make_env():
    env = gym.make(env_id, render_mode=None, tasks_to_complete=[task])  # Or your actual task
    env = FlattenDictWrapper(env)
    return env

env = DummyVecEnv([make_env])

C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\franka_kitchen\../assets/kitchen_franka/kitchen_assets/kitchen_env_model.xml


In [8]:

# The noise objects for DDPG
action_noise = NormalActionNoise(mean=np.zeros(n_actions), sigma=0.1 * np.ones(n_actions))

model = DDPG("MlpPolicy", env, action_noise=action_noise, verbose=1, device=device, buffer_size=5000000)


Using cuda:0 device
Box(-10000000000.0, 10000000000.0, (73,), float32)


In [9]:
# model.learn(total_timesteps=100000, log_interval=10)

In [10]:
# model.save("ddpg_"+task)

In [11]:
# vec_env = model.get_env()

# del model # remove to demonstrate saving and loading

model = DDPG.load("ddpg_"+task)

# obs = vec_env.reset()


Box(-10000000000.0, 10000000000.0, (73,), float32)


In [12]:
# success = False
# run_counter = 0
# while not success:
# 	vec_env.reset()
# 	done = False
# 	counter = 0
# 	while not done:
# 		action, _states = model.predict(obs)
# 		obs, rewards, dones, info = vec_env.step(action)
# 		done = dones[0]
# 		counter += 1
# 	if counter != 280:
# 		success = True
# 	run_counter += 1
# 	print(success, run_counter)


In [13]:
for i in range(10):
	env_eval = make_env()
	obs, _ = env_eval.reset()
	done = False
	ep_reward = 0

	while not done:
		action, _ = model.predict(obs, deterministic=True)
		obs, reward, terminated, truncated, _ = env_eval.step(action)
		done = terminated or truncated
		ep_reward += reward
	print(f"Episode reward: {ep_reward}")

C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\franka_kitchen\../assets/kitchen_franka/kitchen_assets/kitchen_env_model.xml
Episode reward: 0.0
C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\franka_kitchen\../assets/kitchen_franka/kitchen_assets/kitchen_env_model.xml
Episode reward: 0.0
C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\franka_kitchen\../assets/kitchen_franka/kitchen_assets/kitchen_env_model.xml
Episode reward: 0.0
C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\franka_kitchen\../assets/kitchen_franka/kitchen_assets/kitchen_env_model.xml
Episode reward: 0.0
C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\franka_kitchen\../assets/kitchen_franka/kitchen_assets/kitchen_env_model.xml
Episode reward: 0.0
C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\franka_kitchen\../assets/kitchen_franka/kitchen_assets/kitchen_env_model.xml
Episode rewa

ValueError: Error: resource not found via provider or OS filesystem: 'C:\Users\pc\Documents\repos\Gymnasium-Robotics\gymnasium_robotics\envs\assets\kitchen_franka\franka_assets\meshes\collision\link0.stl'

In [ ]:
print("Action space:", env_eval.action_space)
print("Model action space:", model.action_space)

Action space: Box(-1.0, 1.0, (9,), float64)
Model action space: Box(-1.0, 1.0, (9,), float64)
